In [1]:
!pip install -q transformers
#pip install 'transformers[torch]' CPU version of PyTorch
#pip install 'transformers[tf-cpu]' CPU Version of TensorFlow

In [2]:
!pip install -q huggingface_hub

In [3]:
!pip install -q tokenizers

In [4]:
!pip install -q torch

In [7]:
!pip install -q accelerate

In [1]:
!pip install -q chromadb

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

huggingface_api_key  = ''

In [5]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

/opt/homebrew/anaconda3/envs/GenerativeAI/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map='cuda',
    cache_dir='./base_models',
    token=huggingface_api_key
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                         cache_dir='./base_models.',
                                         token=huggingface_api_key)

In [ ]:
## Llama2

def get_llama2_chat_response(prompt, max_new_tokens=100):
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.0001)
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return response

In [48]:
## Llama3 on Groq
from groq import Groq

groq_client = Groq(api_key='')

def get_completion_from_groq(message, client_instance=groq_client, model='', temperator=0):
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": message,
        }
    ]
    response = client_instance.chat.completions.create(
        messages=messages,
        model="llama3-70b-8192",
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        stop=None,
        stream=False,
    )
    return response.choices[0].message.content

In [52]:
## OpenAI
from openai import OpenAI

openai_client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key='',
)

def get_completion_from_messages(message, client_instance=openai_client, model="gpt-4", temperature=0):
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": message,
        }
    ]
    response = client_instance.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

## Vector DB

In [6]:
import chromadb

chroma_client = chromadb.Client()

In [7]:
collection = chroma_client.create_collection(name="my_collection")

In [8]:
collection.add(
    embeddings = [[1,2],[2,1]],
    documents=["This is document 1", "This is document 2"],
    # metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [9]:
results = collection.query(
    query_embeddings = [[4,1]],
    n_results=1
)

In [10]:
results

{'ids': [['id2']],
 'distances': [[4.0]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [['This is document 2']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## Embedding

In [11]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00


In [12]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2',
                                      cache_folder="./base_models"
                                     )

/opt/homebrew/anaconda3/envs/GenAI/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
sentences = ["The man is sitting on the tree", "The girl is eating an apple"]
embeddings = embedding_model.encode(sentences)
embeddings

array([[-0.21421571, -0.05434426,  0.19832453,  0.10691655,  0.28119686,
         0.32432988,  0.1671177 ,  0.20439668, -0.3598284 ,  0.11064065,
        -0.08583799,  0.41642538, -0.3645016 ,  0.85148627,  0.11012043,
         0.92857635, -0.36775345,  0.16180618,  0.44544262,  0.4416718 ,
         0.23353025,  0.02829267,  0.0571992 , -0.70513844,  0.05321491,
        -0.16388686,  0.05863433,  0.0121839 ,  0.49752542, -0.15215246,
         0.29764974, -0.07951561,  0.7024071 ,  0.42614806, -0.4737264 ,
        -0.2805278 , -0.45465115, -0.3122342 ,  0.10630064,  0.10367037,
        -0.56606936,  0.16192491,  0.14529684,  0.7097154 , -0.4996835 ,
         0.23867501, -0.05783312, -0.4678942 , -0.18654986,  0.14146481,
        -0.1144841 , -0.13624042,  0.0772586 , -0.41289452,  0.5603255 ,
         0.3067136 ,  0.00458807, -0.04948796,  0.6024598 ,  0.2552421 ,
         0.00164488, -0.7260487 ,  0.34768158, -0.22945112,  0.00646364,
        -0.28977212, -0.36313832, -0.4536401 ,  0.0

## RAG (Retreival Augmented Generation)

In [15]:
!pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00


In [16]:
import PyPDF2

In [17]:
# creating a pdf reader object
reader = PyPDF2.PdfReader('integrated_en.pdf')

In [18]:
print(len(reader.pages))

65


In [19]:
# print the text of the first page
print(reader.pages[0].extract_text())

Integrated 
Report 2022
Integrated Report 2022
Fiscal year ended March 31, 2022


In [20]:
pages = [page.extract_text() for page in reader.pages]

In [21]:
document = '\n'.join(pages)

In [22]:
def get_overlapped_chunks(textin, chunksize, overlapsize):
    return [textin[a:a+chunksize] for a in range(0,len(textin), chunksize-overlapsize)]


In [23]:
chunks = get_overlapped_chunks(document, 2000, 100)

In [24]:
len(chunks)

210

In [25]:
chunks[0]

'Integrated \nReport 2022\nIntegrated Report 2022\nFiscal year ended March 31, 2022\nTOYOTA MOTOR CORPORATION INTEGRATED REPORTMessage from \nthe PresidentThe Source of Our \nValue Creation:\n What Makes Us ToyotaValue Creation Story: \nWorking toward the Mobility \nSociety of the FutureBusiness Foundations \nfor Value CreationCorporate Data\n5 Our Founding Spirit   The Toyoda Principles and Toyota Philosophy   Toyota Production System (TPS)   Toyota and Sports\nThe Toyoda Principles and Toyota Philosophy\nWith an eye toward “Producing Happiness \nfor All” - Selections from the Q&A session of \nToyota’s 1H/2Q financial results briefing\nIn 1935, five years after the passing of Sakichi Toyoda, the \nCompany had grown to more than 10,000 employees as the \nautomotive business ramped up. The Toyoda Principles were \ncompiled at this time to convey Sakichi’s teachings to all employees \nand provide guidelines for all aspects of their work. \n The top management of Toyota that took over fro

In [26]:
chunks[1]

'ta Group \ncompany and serve as \nguidelines for all employees.\nToyota’s mission since its foundation\nProducing Happiness for All\nBorn into a family of poor farmers, Sakichi Toyoda built the bedrock of today’s Toyota by inventing the Toyoda Automatic Loom by \nhimself. Abandoning the easy path left by his father, Kiicihro Toyoda took on the challenge of making cars. Many at the time said it \nwas impossible.\n Their passion was carried on by those who worked with them, shaping the Toyota we know today. What they truly wanted to make \nwas a sense of happiness for any customer who used their products, as well as happiness for every person involved in the work \nrelated to those products. The core of this aspiration was the idea of producing happiness for all. \n However, during Toyota’s long history, there was a brief time when we turned our focus to numbers and gave less thought to people. \nPrimarily due to our rapid expansion in the late 20th century, we faced many problems, incl

## Embedding

In [27]:
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = 'base_models')
chunk_embeddings = embedding_model.encode(chunks)
chunk_embeddings.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(210, 768)

In [28]:
collection = chroma_client.create_collection(name="rag_llama2-1")

In [29]:
collection.add(
    embeddings = chunk_embeddings,
    documents=chunks,
    ids= [str(i) for i in range(len(chunks))]
)

## Retrieval

In [30]:
def retrieve_vector_db(query, n_results=3):
  results = collection.query(
  query_embeddings = embedding_model.encode(query).tolist(),
  n_results=n_results
  )
  return results['documents']

In [31]:
query = "Who is Toyoda?"
retrieved_results = retrieve_vector_db(query)

In [32]:
retrieved_results[0]

['ation \n worsened further. As the Company’s business \nresults showed no sign of improvement, \n labor-management negotiations with the Toyota \nMotor Co., Ltd. labor union deteriorated into a \nprotracted dispute. During collective bargaining \nthat April, the Company made reconstruction \nproposals centered on job cuts that the labor \nunion could not accept, and the dispute \n continued for another month and a half until \na\xa0memorandum was finally signed in June. \n Accepting responsibility for the labor disputes, \nKiichiro Toyoda resigned as president of the \nCompany in May 1950. In March 1952, he agreed \nto make his much-awaited return to the position, \nbut, before he could do so, he passed away at \nthe age of 57. Nevertheless, his aspirations were \nkept alive by his colleagues, who persevered with \npurely home-grown technologies as other \nJapanese automakers were forming technology \nalliances with U.S. and European manufacturers. \nThese efforts led to the 1955 laun

In [33]:
context = '\n\n'.join(retrieved_results[0])
print(context)

ation 
 worsened further. As the Company’s business 
results showed no sign of improvement, 
 labor-management negotiations with the Toyota 
Motor Co., Ltd. labor union deteriorated into a 
protracted dispute. During collective bargaining 
that April, the Company made reconstruction 
proposals centered on job cuts that the labor 
union could not accept, and the dispute 
 continued for another month and a half until 
a memorandum was finally signed in June. 
 Accepting responsibility for the labor disputes, 
Kiichiro Toyoda resigned as president of the 
Company in May 1950. In March 1952, he agreed 
to make his much-awaited return to the position, 
but, before he could do so, he passed away at 
the age of 57. Nevertheless, his aspirations were 
kept alive by his colleagues, who persevered with 
purely home-grown technologies as other 
Japanese automakers were forming technology 
alliances with U.S. and European manufacturers. 
These efforts led to the 1955 launch of the 
Toyopet Crown, 

## Augmented Generation

In [34]:
prompt = f'''
Give answer for the question strictly based on the context provided.

Question: {query}

Context : {context}
'''

In [35]:
prompt

'\nGive answer for the question strictly based on the context provided.\n\nQuestion: Who is Toyoda?\n\nContext : ation \n worsened further. As the Company’s business \nresults showed no sign of improvement, \n labor-management negotiations with the Toyota \nMotor Co., Ltd. labor union deteriorated into a \nprotracted dispute. During collective bargaining \nthat April, the Company made reconstruction \nproposals centered on job cuts that the labor \nunion could not accept, and the dispute \n continued for another month and a half until \na\xa0memorandum was finally signed in June. \n Accepting responsibility for the labor disputes, \nKiichiro Toyoda resigned as president of the \nCompany in May 1950. In March 1952, he agreed \nto make his much-awaited return to the position, \nbut, before he could do so, he passed away at \nthe age of 57. Nevertheless, his aspirations were \nkept alive by his colleagues, who persevered with \npurely home-grown technologies as other \nJapanese automakers

In [41]:
print(get_completion_from_groq(prompt))
#print(get_llama2_chat_response(prompt, max_new_tokens=500))

In the provided context, Toyoda refers to the family name of the founders of Toyota Motor Co., Ltd. Specifically, Kiichiro Toyoda is mentioned as the president of the company who resigned due to labor disputes and Sakichi Toyoda is mentioned as the father of Kiichiro Toyoda and the inventor of the Toyoda automatic loom. The Spirit of Sakichi and Kiichiro Toyoda is a guiding philosophy for Toyota Group companies. The surname Toyoda is written with the characters 豊田, which literally mean "fertile rice paddies."


## RAG

In [45]:
# 1. Retrieve Vector DB
query = "What is Toyoda's Principles?"

retrieved_results = retrieve_vector_db(query, n_results=5)
context = '\n\n'.join(retrieved_results[0])

# 2. Augment Context
prompt = f'''
Give answer for the question strictly based on the context provided. Keep answers short and to the point.

Question: {query}

Context : {context}
'''

# 3. Pass to LLM for Generation
print(get_completion_from_groq(prompt))

The Toyoda Principles, also known as the Five Main Principles of Toyoda, are:

1. We unite as one team regardless of rank in order to contribute to our people, society, and communities.
2. We develop and learn from outstanding ideas and cutting-edge technologies across the world.
3. We focus on work that is value-adding, with integrity and practicality, by avoiding superficial matters.
4. We build a sense of community and promote the personal growth of our people while valuing mutual trust and equal partnership with our stakeholders.
5. We show humility for the support of our business by our valued stakeholders and society while also respecting the diversity of the world.


In [47]:
# 1. Retrieve Vector DB
query = "What are units sales in Europe for Corolla in 2022?"

retrieved_results = retrieve_vector_db(query, n_results=5)
context = '\n\n'.join(retrieved_results[0])

# 2. Augment Context
prompt = f'''
Give answer for the question strictly based on the context provided. Keep answers short and to the point.

Question: {query}

Context : {context}
'''

# 3. Pass to LLM for Generation
print(get_completion_from_groq(prompt))

The answer to the question "What are units sales in Europe for Corolla in 2022?" is 186,000 units.


In [53]:
## OpenAI 
print(get_completion_from_messages(prompt))

The unit sales for Corolla in Europe for 2022 were 186 thousand units.
